### Load The necessary library

In [2]:
library(tidyverse)
library(stringr)
library(dplyr)

-- Attaching packages --------------------------------------- tidyverse 1.3.1 --

v ggplot2 3.3.6     v purrr   0.3.4
v tibble  3.1.7     v dplyr   1.0.9
v tidyr   1.2.0     v stringr 1.4.0
v readr   2.1.2     v forcats 0.5.1

-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()



### Check the structure of each dataset

In [41]:
dataset_list<-c('raw_bike_sharing_system.csv','raw_cities_weather_forecast.csv','raw_worldcities.csv','raw_seoul_bike_sharing.csv')

In [44]:
# loop thorugh each dataset to see every datasaets structure
for(dataset_names in dataset_list){
    datasets<-read.csv(dataset_names)
    print(str(datasets))
}

Warning message in file(file, "rt"):
"cannot open file 'raw_bike_sharing_system.csv': No such file or directory"


ERROR: Error in file(file, "rt"): cannot open the connection


### Convert all the dataset collumn name into uppercase and replace dot symbol with underscore

In [ ]:
for(dataset_name in dataset_list){
    dataset <- read.csv(dataset_name)
    names(dataset)<- toupper(names(dataset))
    names(dataset)<-str_replace_all(names(dataset),"[.]","_")
    write.csv(dataset,dataset_name,row.names=FALSE)
    file.rename(dataset_name,gsub("raw_","",dataset_name))
    }

### load the bike sharing system dataset

In [45]:
bike_sharing_df <- read.csv("bike_sharing_system.csv")
head(bike_sharing_df,10)

,COUNTRY,CITY,NAME,SYSTEM,OPERATOR,LAUNCHED,DISCONTINUED,STATIONS,BICYCLES,DAILY_RIDERSHIP
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,Albania,Tirana[5],Ecovolis,,,March 2011,,8,200,
2,Argentina,Buenos Aires[6][7],Ecobici,Serttel Brasil[8],Bike In Baires Consortium.[9],2010,,400,4000,21917
3,Argentina,Mendoza[10],Metrobici,,,2014,,2,40,
4,Argentina,Rosario,Mi Bici Tu Bici[11],,,2 December 2015,,47,480,
5,Argentina,"San Lorenzo, Santa Fe",Biciudad,Biciudad,,27 November 2016,,8,80,
6,Australia,Melbourne[12],Melbourne Bike Share,PBSC & 8D,Motivate,June 2010,30 November 2019[13],53,676,
7,Australia,Melbourne[12],oBike,4 Gen. oBike,,July 2017,July 2018,dockless,1250,
8,Australia,Brisbane[14][15],CityCycle,3 Gen. Cyclocity,JCDecaux,September 2010,,150,2000,
9,Australia,Sydney,oBike,4 Gen. oBike,,July 2017,July 2018,dockless,1250,


## Check every collumn for reference link ([9])

In [46]:
find_reference <- function(strings) grepl("\\[[A-z0-9]+\\]", strings)
find_character <- function(strings) grepl("[^0-9]", strings)
a<-bike_sharing_df %>% select(COUNTRY) %>% filter(find_reference(COUNTRY)) %>% slice(0:5)
b<-bike_sharing_df %>% select(CITY) %>% filter(find_reference(CITY)) %>% slice(0:5)
c<-bike_sharing_df %>% select(NAME) %>% filter(find_reference(NAME)) %>% slice(0:5)
d<-bike_sharing_df %>% select(SYSTEM) %>% filter(find_reference(SYSTEM)) %>% slice(0:5)
e<-bike_sharing_df %>% select(OPERATOR) %>% filter(find_reference(OPERATOR)) %>% slice(0:5)
f<-bike_sharing_df %>% select(LAUNCHED) %>% filter(find_reference(LAUNCHED)) %>% slice(0:5)
g<-bike_sharing_df %>% select(DISCONTINUED) %>% filter(find_reference(DISCONTINUED)) %>% slice(0:5)
h<-bike_sharing_df %>% select(STATIONS) %>% filter(find_reference(STATIONS)) %>% slice(0:5)
i<-bike_sharing_df %>% select(BICYCLES) %>% filter(find_character(BICYCLES)) %>% slice(0:5)
j<-bike_sharing_df %>% select(DAILY_RIDERSHIP) %>% filter(find_reference(DAILY_RIDERSHIP)) %>% slice(0:5)
a
combined<- data.frame(b,c,d,e,f,g,h,i,j)
combined

COUNTRY
<chr>


CITY,NAME,SYSTEM,OPERATOR,LAUNCHED,DISCONTINUED,STATIONS,BICYCLES,DAILY_RIDERSHIP
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
Tirana[5],Mi Bici Tu Bici[11],Serttel Brasil[8],Bike In Baires Consortium.[9],2014 [66],30 November 2019[13],70 (2021)[20],1790 (2019)[21],2800[18]
Buenos Aires[6][7],Citybike Wien[17],EasyBike[64],The Metropolitan Area of Aburra Valley[59],2015 [67],2020[41],24[121],4200 (2021),7010[23]
Mendoza[10],Velo Antwerp[22],4 Gen.[72],Smovengo[113],2018 [68],2020[77],15 & dockless [130],4115[25],"4,700[74][non-primary source needed]"
Melbourne[12],Rekola[95],3 Gen. SmooveKey[120],Batumi Avtotransporti[121],2015 [69],October 2012 [85][86][87] Copenhagen municipality changed its mind and a new version was introduced in late 2013.[88],320[179],7270 (regular) 2395 (electric)[38],376[97]
Melbourne[12],BatumVelo[119],3 Gen. Smoove[147][148][149][145],Freie Lastenräder [127],2017 [70],2010 [93],dockless[210],310[65],380[97]


### Remove Reference link from all collum

In [47]:
remove_ref <- function(strings){
results<-str_replace_all(strings,"\\[[\\w]+\\]"," ")    
}
cleaned_df<-bike_sharing_df %>% mutate(CITY=remove_ref(CITY), NAME=remove_ref(NAME), SYSTEM=remove_ref(SYSTEM),
OPERATOR=remove_ref(OPERATOR), LAUNCHED=remove_ref(LAUNCHED), DISCONTINUED=remove_ref(DISCONTINUED), 
STATIONS=remove_ref(STATIONS), BICYCLES=remove_ref(BICYCLES),DAILY_RIDERSHIP=remove_ref(DAILY_RIDERSHIP))

### Check again to see if the reference link removed

In [48]:
cleaned_df %>%
    select(CITY,NAME,SYSTEM,OPERATOR,LAUNCHED,DISCONTINUED,STATIONS,BICYCLES,DAILY_RIDERSHIP) %>%
    filter(find_reference(CITY) | find_reference(NAME) | find_reference(SYSTEM) | find_reference(OPERATOR) |
    find_reference(LAUNCHED) | find_reference(DISCONTINUED) | find_reference(STATIONS) | find_character(BICYCLES) |
    find_reference(DAILY_RIDERSHIP) )%>%
    slice(0:10)

CITY,NAME,SYSTEM,OPERATOR,LAUNCHED,DISCONTINUED,STATIONS,BICYCLES,DAILY_RIDERSHIP
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
Different locations,dBlue-bike,Blue-bike,Blue-Mobility,1 May 2011,,70 (2021),1790 (2019),
Antwerp,Velo Antwerp,3 Gen. Clear CC,Service2Cities (subsidiary of Clear Channel Belgium),9 June 2011,,305 (2021),4200 (2021),7010
Brussels,Villo!,3 Gen. Cyclocity,JCDecaux,19 May 2009 (earlier system since 2006),,346,4115,
Montreal,BIXI Montréal,PBSC & 8D,,2009,,794,7270 (regular) 2395 (electric),
"Limassol, Nicosia District",Bike in Action,3 Gen. Smoove,,,,27,310,
Prague,Freebike,,HomePort,,,,500,
Prague 7,Ofo,4 Gen. Ofo,Ofo,2017,Discontinued,,,
Prostejov,Nextbike,3 Gen. nextbike,Nextbike,2019,,,180,
Ostrava,Nextbike,3 Gen. nextbike,Nextbike,2019,,,600,


### extract the numeric value from BICYCLES Collumn

In [49]:
extract_num<-function(columns){
    extract <- str_extract(columns,"[0-9]+")
    extracted_value<-as.numeric(extract)
    return(extracted_value)
}
final_result<-cleaned_df %>% mutate(BICYCLES=extract_num(BICYCLES))

### Check again to see if it was removed

In [50]:
final_result %>%
    select(BICYCLES) %>%
        filter(find_character(BICYCLES))

BICYCLES
<dbl>


In [51]:
write.csv(final_result,"bike_sharing_system.csv",row.names=FALSE)